In [1]:
pwd

'/mmfs1/data/bergerar/BC-MRI-ADHD/Code'

In [2]:
import os 
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from make_models import get_MRI_VAE_3D,get_MRI_CVAE_3D
from tqdm import tqdm 

2022-08-30 22:22:00.319219: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [3]:
stuff = np.load('../Assets/brain_array-440.npz')
print(list(stuff.keys()))

data = stuff['data']
control = stuff['controls']
patients = stuff['patients']
n = data.shape[0]
print(data.shape)
data[patients,:,:,:].shape
data[patients].shape


['data', 'controls', 'patients']
(440, 64, 64, 64)


(179, 64, 64, 64)

In [ ]:
batch_size = 32
encoder, decoder, vae = get_MRI_VAE_3D(input_shape=(64,64,64,1),
                   latent_dim=2,
                   batch_size = batch_size,
                   disentangle=False,
                   gamma=1,
                   kernel_size = 3,
                   filters = 16,
                   intermediate_dim = 128,
                   opt=None)

In [ ]:
# Training
#for epoch in epochs
for i in tqdm(range(5)):
    batch_idx = np.random.randint(low=0,high=n,size=batch_size)
    data_batch = data[batch_idx,:,:,:]
    vae.train_on_batch(data_batch)

In [ ]:
prediction = vae.predict(data_batch)

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(data_batch[0,:,:,32])
plt.title('input')
plt.subplot(1,2,2)
plt.imshow(prediction[0,:,:,32,0])
plt.title('prediction')

In [ ]:
cvae, z_encoder, s_encoder, cvae_decoder = get_MRI_CVAE_3D(input_shape=(64,64,64,1),
                    latent_dim=2,
                    beta=1,
                    disentangle=False,
                    gamma=1,
                    bias=True,
                    batch_size = 64,
                    kernel_size = 3,
                    filters = 32,
                    intermediate_dim = 128,
                    opt=None)

# CVAE takes in [patients,controls]

In [ ]:
data_batch_patients = data[np.random.randint(low=0,high=n,size=batch_size),:,:,:]
data_batch_control = data[np.random.randint(low=0,high=n,size=batch_size),:,:,:]

In [ ]:
for i in tqdm(range(5)):
    cvae.train_on_batch([data_batch_patients,data_batch_control])

In [ ]:
predictions = cvae.predict([data_batch_patients,data_batch_control])

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(data_batch_patients[0,:,:,32])
plt.title('input')
plt.subplot(1,2,2)
plt.imshow(predictions[0][0,:,:,32,0])
plt.title('prediction')